In [ ]:
!pip install --user --upgrade catboost
!pip install flask-ngrok
import catboost
print(catboost.__version__)

Requirement already up-to-date: catboost in /root/.local/lib/python3.7/site-packages (0.26)
0.26


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
import pandas as pd
import datetime 
 
path = f'drive/MyDrive/odds/{"{:02d}".format(datetime.datetime.today().day)}.{"{:02d}".format(datetime.datetime.today().month)}/'
print(path)
cnx = sqlite3.connect(path+"data.sqlite3")
df = pd.read_sql_query("select * from results where good is not null and good!='error' and next is not null", cnx)
# 41778
print(len(df))
df.head(5)

drive/MyDrive/odds/11.07/
73346


,id,next,time,unix,timeleft,sport,championat,league,match,event,status,coefficient,percent,result,good,url,marathon,bet1x,leonbets,winline,pred,prob,logloss,pred_next,prob_next,logloss_next,pred_auc,prob_auc,logloss_auc,pred_prec,prob_prec,logloss_prec,day
0,SPnBjSBCHAytyP7ALzsH9ONNYoc=,94XoYtVZJF3LtIC1KWaMPGbymsE=,17 Jan 2021 20:00,1610913600,9414.643,Soccer,Brazil,Serie B,Vitoria - Chapecoense-SC,EH +2,2,11.46,38,0:0,0,https://www.oddsportal.com/soccer/brazil/serie...,11.00,11.00,16.25,0.0,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
1,XPKq1mxiL8km83tvX8wf1jG-M_w=,rLMPf0DYELthOqG-M_e9U28DStI=,17 Jan 2021 21:10,1610917800,13614.643,Soccer,Argentina,Primera Nacional,Deportivo Moron - Villa Dalmine,O/U 3,Over,3.99,36,2:1,0,https://www.oddsportal.com/soccer/argentina/pr...,4.05,4.04,0.00,0.0,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
2,ly2sLzTv9FTKGiyK7EiW0msvP9Y=,94XoYtVZJF3LtIC1KWaMPGbymsE=,17 Jan 2021 20:00,1610913600,9414.643,Soccer,Brazil,Serie B,Vitoria - Chapecoense-SC,EH +1,2,4.99,35,0:0,0,https://www.oddsportal.com/soccer/brazil/serie...,5.10,5.10,5.80,0.0,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
3,YD1ikWg99OtFmVdjzfKXI-dLxhs=,rLMPf0DYELthOqG-M_e9U28DStI=,17 Jan 2021 21:10,1610917800,13614.642,Soccer,Argentina,Primera Nacional,Deportivo Moron - Villa Dalmine,AH +1,2,5.00,33,2:1,0,https://www.oddsportal.com/soccer/argentina/pr...,5.10,5.10,0.00,0.0,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
4,djkuJQL11W1W4s_qWg1XctxVnCw=,rLMPf0DYELthOqG-M_e9U28DStI=,17 Jan 2021 21:10,1610917800,13614.642,Soccer,Argentina,Primera Nacional,Deportivo Moron - Villa Dalmine,EH +1,2,6.35,32,2:1,0,https://www.oddsportal.com/soccer/argentina/pr...,6.00,6.00,7.20,0.0,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None


In [ ]:
cnx = sqlite3.connect(path+"next.sqlite3")
nexts = pd.read_sql_query("select id, odd1, oddX, odd2 from next", cnx)
print(len(nexts))
nexts.head(5)

122457


,id,odd1,oddX,odd2
0,alGLmMgYw-Hp3gQnEVMMJF0JM7A=,2.55,2.98,2.94
1,Ui8s2L_xObIrJMt2tsgtgKwdzLY=,2.11,2.85,4.14
2,rLMPf0DYELthOqG-M_e9U28DStI=,2.67,2.94,2.83
3,n6QWccMa5ZYJ0GIX9SzaJ85c_wA=,2.27,3.21,3.17
4,tqRmfzLztjyZamWYu1uKn_wwm4k=,2.13,3.14,3.59


In [ ]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))
 
data = join_df(df, nexts, "next", "id")
data = data[data.odd1.notnull()]
 
import numpy as np
data = data.replace('', np.nan)
column_names = ["id","next", "unix", "sport", "championat", "league", "match", "event", "status", "coefficient", "percent", "result", "good", "odd1","oddX","odd2"]
 
 
data.sport = data.sport.str.strip().str.lower()
data.championat = data.championat.str.strip().str.lower()
data.league = data.league.str.strip().str.lower()
data.match = data.match.str.strip().str.lower()
data.event = data.event.str.strip().str.lower()
 
 
 
df = data.copy()
df = df[column_names]
df['coefficient'] = df['coefficient'].astype(float)
df['percent'] = df['percent'].astype(float)
df['unix'] = df['unix'].astype(float)
df['good'] = df['good'].astype(float)
df['odd1'] = df['odd1'].astype(float)
df['oddX'] = df['oddX'].astype(float)
df['odd2'] = df['odd2'].astype(float)
df['won'] =["won" if g>=1.0 else "lost" for g in df.good]
 
df["ev"] = df['event'].str.split(r'\s+', expand=True)[0]
df["ent"] = df['event'].str.split(r'\s+', expand=True)[1].astype(float)
 
w = len(df[df['won'] == 'won'])
l = len(df[df['won'] == 'lost'])
 
print(f'Note: initialy data has {w} - won and {l} {w*100/(w+l)}% ({w}) won and {l*100/(w+l)}% ({l}) lost')
 
df  =  df[['sport','championat','league','ev', 'ent', 'status','coefficient','percent','odd1','oddX','odd2','won']]
df.ent = df.ent.fillna(value=0.0)
 
X = df.copy()
 
y = X.pop('won')
cat_features = np.where(X.dtypes != np.float)[0]
cat_features

Note: initialy data has 20310 - won and 51118 28.434227473819792% (20310) won and 71.56577252618021% (51118) lost


array([0, 1, 2, 3, 5])

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import Pool
 
data = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_validation, y_train, y_validation = data
 
train_pool = Pool(
    data=X_train, 
    label=y_train, 
    cat_features=cat_features
)
 
validation_pool = Pool(
    data=X_validation, 
    label=y_validation, 
    cat_features=cat_features
)

In [ ]:
from catboost import CatBoostClassifier
 
model = CatBoostClassifier()
 
model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=10,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.086075
0:	learn: 0.6556989	test: 0.6554477	best: 0.6554477 (0)	total: 162ms	remaining: 2m 42s
10:	learn: 0.5176547	test: 0.5158778	best: 0.5158778 (10)	total: 961ms	remaining: 1m 26s
20:	learn: 0.4954697	test: 0.4928561	best: 0.4928561 (20)	total: 1.68s	remaining: 1m 18s
30:	learn: 0.4864876	test: 0.4832630	best: 0.4832630 (30)	total: 2.4s	remaining: 1m 15s
40:	learn: 0.4812257	test: 0.4776337	best: 0.4776337 (40)	total: 3.2s	remaining: 1m 14s
50:	learn: 0.4774981	test: 0.4738345	best: 0.4738345 (50)	total: 3.97s	remaining: 1m 13s
60:	learn: 0.4749753	test: 0.4715010	best: 0.4715010 (60)	total: 4.78s	remaining: 1m 13s
70:	learn: 0.4731154	test: 0.4700379	best: 0.4700379 (70)	total: 5.53s	remaining: 1m 12s
80:	learn: 0.4718950	test: 0.4692760	best: 0.4692760 (80)	total: 6.28s	remaining: 1m 11s
90:	learn: 0.4701595	test: 0.4677445	best: 0.4677445 (90)	total: 7.1s	remaining: 1m 10s
100:	learn: 0.4684887	test: 0.4664628	best: 0.4664628 (100)	total: 7.84s	remaining: 1

In [ ]:
import numpy as np

def get_df(id, sport, championat, league, event, status, coefficient, percent, odd1,oddX,odd2):
    d = {'id': [id],'sport': [sport], 'championat': [championat], 'league': [league], 'event': [event], 'status': [status], 'coefficient': [coefficient], 'percent': [percent], 'odd1': [odd1], 'oddX': [oddX], 'odd2': [odd2]}
    df = pd.DataFrame(data=d)
    df = df.replace('', np.nan)
    df.sport = df.sport.str.strip().str.lower()
    df.championat = df.championat.str.strip().str.lower()
    df.league = df.league.str.strip().str.lower()
    df.event = df.event.str.strip().str.lower()
    df['coefficient'] = df['coefficient'].astype(float)
    df['percent'] = df['percent'].astype(float)
    df['odd1'] = df['odd1'].astype(float)
    df['oddX'] = df['oddX'].astype(float)
    df['odd2'] = df['odd2'].astype(float)
    if event == "1X2" or event == "DNB" or event == "DC" or event == "BTS" or event == "DC":
        df["ev"] = event
        df["ent"] = 0.0    
    else:
        df["ev"] = df['event'].str.split(r'\s+', expand=True)[0]
        df["ent"] = df['event'].str.split(r'\s+', expand=True)[1].astype(float)
  
    df  =  df[['sport','championat','league','ev', 'ent', 'status','coefficient','percent','odd1','oddX','odd2']]
    df.ent = df.ent.fillna(value=0.0)
 
    return df

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken 1o4kaWmDPuTPn9ttACi20sykhni_21xtSpkXK2o8G6ZHJBXji

--2021-06-07 06:30:25--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.230.235.205, 52.55.225.227, 34.197.154.58, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.230.235.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  17.6MB/s    in 0.8s    

2021-06-07 06:30:26 (17.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd
import time
 
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
 
@app.route('/', methods=['GET', 'POST'])
def add_message():
    id = request.args.get('id')
    sport = request.args.get('sport')
    championat = request.args.get('championat')
    league = request.args.get('league')
    event = request.args.get('event')
    status = request.args.get('status')
    coefficient = request.args.get('coefficient')
    percent = request.args.get('percent')
    odd1 = request.args.get('odd1')
    oddX = request.args.get('oddX')
    odd2 = request.args.get('odd2')
    
    df = get_df(id, sport, championat, league, event, status, coefficient, percent,odd1,oddX,odd2)
    prediction = model.predict(df)
    prob = model.predict_proba(df)
    response = id + "<===>" + prediction[0] + "<===>" + str(prob[0][1]) + "<===>" + str(model.best_score_["validation"]["Logloss"])
    print(f'{time.strftime("%d %m %Y, %H:%M", time.localtime())}   id:{id},sport:{sport},championat:{championat},league:{league},event:{event},status:{status},coefficient:{coefficient},percent:{percent}, odd1: {odd1},oddX: {oddX},odd2: {odd2}; ---> {prediction[0]}')
    print(response)
    return response
 
@app.route('/test', methods=['GET', 'POST'])
def test():
    print(f'Connected!')
    return "default"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://885fceb31432.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jun/2021 06:32:32] "GET /test HTTP/1.1" 200 -


Connected!


...

07 06 2021, 07:17   id:p1cSYRcqybjEBRvqtLsdb-NTDjo=,sport:Soccer,championat:World,league:Friendly International,event:EH +1,status:2,coefficient:11.08,percent:18, odd1: 2.13,oddX: 3.13,odd2: 3.62; ---> lost
p1cSYRcqybjEBRvqtLsdb-NTDjo=<===>lost<===>0.04087484202800032<===>0.4032753665215197


127.0.0.1 - - [07/Jun/2021 07:17:28] "GET /?championat=Brazil&coefficient=2.31&event=AH+0&id=B9P2BxYIRwFkFwlus-uenLfAec8%3D&league=Serie+C&odd1=1.91&odd2=4.5&oddX=2.84&percent=18&sport=Soccer&status=2 HTTP/1.1" 200 -


07 06 2021, 07:17   id:B9P2BxYIRwFkFwlus-uenLfAec8=,sport:Soccer,championat:Brazil,league:Serie C,event:AH 0,status:2,coefficient:2.31,percent:18, odd1: 1.91,oddX: 2.84,odd2: 4.5; ---> lost
B9P2BxYIRwFkFwlus-uenLfAec8=<===>lost<===>0.4137207065510219<===>0.4032753665215197


127.0.0.1 - - [07/Jun/2021 07:32:16] "GET /test HTTP/1.1" 200 -


Connected!
